<a href="https://colab.research.google.com/github/eliasantoro99/AI-REPO/blob/main/training_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# ==================== STEP 1: IMPORT LIBRERIE ==========================
from roboflow import Roboflow                    # Per connettersi a Roboflow e scaricare dataset
import torch                                     # Libreria PyTorch base
import torch.nn as nn                            # Per definire layer e architetture delle reti
import torch.optim as optim                      # Per ottimizzatori come Adam
from torchvision import datasets, transforms, models  # Per dataset, trasformazioni immagini e modelli predefiniti
from torch.utils.data import DataLoader          # Per gestire i batch di dati durante il training
from tqdm import tqdm                            # Per visualizzare una barra di avanzamento
import os                                        # Per lavorare con file system (percorso file, ecc.)

# ==================== STEP 2: SCARICAMENTO DATASET ROBOFLOW ==========================
print("⬇️ Scarico il dataset da Roboflow...")

# Inizializza client Roboflow usando la tua API key (qui pubblica)
rf = Roboflow(api_key="GAQ2cbOSPLavcNiilrPr")

# Seleziona il workspace e progetto da cui scaricare il dataset
project = rf.workspace("roboanime").project("roboanimeproject")

# Scarica la versione 1 del dataset nel formato "folder" (organizzato in sottocartelle)
dataset = project.version(2).download("folder")

# Memorizza il percorso locale del dataset scaricato
data_dir = dataset.location
print(f"✅ Dataset salvato in: {data_dir}")

# ==================== STEP 3: PARAMETRI DI CONFIGURAZIONE ==========================
batch_size = 32              # Quante immagini usare in ogni batch di training
max_epochs = 20              # Numero massimo di epoche (ripetizioni su tutto il dataset)
patience = 5                 # Numero di epoche da aspettare senza miglioramento prima dello stop
lr = 1e-4                    # Learning rate: quanto velocemente il modello impara
best_model_path = "best_model.pt"  # Dove salvare il miglior modello

# Conta il numero di classi leggendo il numero di sottocartelle nella directory "train"
num_classes = len(os.listdir(os.path.join(data_dir, "train")))

# Se c'è una GPU disponibile, usa quella, altrimenti usa la CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"📊 Numero classi: {num_classes}")
print(f"🖥️  Esecuzione su: {device}")

# ==================== STEP 4: DEFINIZIONE TRASFORMAZIONI IMMAGINI ==========================
# Le immagini devono essere tutte ridimensionate a 224x224 (formato richiesto da ResNet18)
# Normalize serve a riportare i valori RGB all'intervallo [-1, 1] come atteso dai modelli pre-addestrati su ImageNet

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),           # Resize immagine
        transforms.RandomHorizontalFlip(),       # Flip orizzontale casuale per data augmentation
        transforms.ToTensor(),                   # Converti immagine in tensore
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Normalizza secondo le statistiche di ImageNet
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

# ==================== STEP 5: CARICAMENTO DATI E DATASET ==========================
print("📂 Carico immagini di training e validazione...")

# Crea dataset da cartelle locali, applicando le trasformazioni
train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=data_transforms['train'])
val_dataset   = datasets.ImageFolder(os.path.join(data_dir, "valid"), transform=data_transforms['val'])

# Crea dataloader per leggere i dati a batch (in modo più efficiente)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  # shuffle solo per train
val_loader   = DataLoader(val_dataset, batch_size=batch_size)

print(f"✅ Immagini train: {len(train_dataset)} | valid: {len(val_dataset)}")

# ==================== STEP 6: COSTRUZIONE MODELLO RESNET18 ==========================
print("🧠 Creo modello ResNet18 pre-addestrato...")

# Carica un modello ResNet18 pre-addestrato su ImageNet
model = models.resnet18(pretrained=True)

# Sostituisci l’ultimo layer per adattarlo al numero delle classi del tuo dataset
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Sposta il modello su GPU (o CPU)
model = model.to(device)

# ==================== STEP 7: FUNZIONE DI PERDITA E OTTIMIZZATORE ==========================
criterion = nn.CrossEntropyLoss()              # Funzione di loss per classificazione multi-classe
optimizer = optim.Adam(model.parameters(), lr=lr)  # Ottimizzatore Adam

# ==================== STEP 8: EARLY STOPPING SETUP ==========================
best_val_loss = float('inf')   # Per tenere traccia della loss migliore vista finora
epochs_no_improve = 0          # Conteggio di quante epoche senza miglioramento

# ==================== STEP 9: CICLO DI TRAINING CON EARLY STOPPING ==========================
print("🚀 Inizio training...\n")
for epoch in range(1, max_epochs + 1):
    print(f"\n🔁 Epoch {epoch}/{max_epochs}")

    for phase in ['train', 'val']:
        model.train() if phase == 'train' else model.eval()  # Modalità training o evaluation

        running_loss = 0.0
        running_corrects = 0
        total_samples = 0

        # Usa il dataloader corrispondente alla fase
        dataloader = train_loader if phase == 'train' else val_loader

        # Loop su tutti i batch della fase corrente
        for inputs, labels in tqdm(dataloader, desc=phase, leave=False):
            inputs, labels = inputs.to(device), labels.to(device)  # Sposta dati su GPU/CPU
            optimizer.zero_grad()  # Azzeramento dei gradienti

            with torch.set_grad_enabled(phase == 'train'):  # Solo nel training fai il backward
                outputs = model(inputs)            # Predizione
                _, preds = torch.max(outputs, 1)   # Classe predetta
                loss = criterion(outputs, labels)  # Calcolo della perdita

                if phase == 'train':
                    loss.backward()                # Backpropagation
                    optimizer.step()               # Aggiorna pesi

            # Accumula risultati per calcolare accuracy e loss medi
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total_samples += inputs.size(0)

        # Calcolo di loss e accuracy medi su tutta l'epoca
        epoch_loss = running_loss / total_samples
        epoch_acc = running_corrects.double() / total_samples
        print(f"📊 {phase.capitalize()} Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}")

        # ---------------- EARLY STOPPING SULLA VALIDAZIONE ----------------
        if phase == 'val':
            if epoch_loss < best_val_loss:
                best_val_loss = epoch_loss
                torch.save(model.state_dict(), best_model_path)  # Salva modello migliore
                print("💾 Model migliorato! Salvato.")
                epochs_no_improve = 0  # Reset del contatore
            else:
                epochs_no_improve += 1
                print(f"⏳ Nessun miglioramento ({epochs_no_improve}/{patience})")
                if epochs_no_improve >= patience:
                    print("🛑 Early stopping attivato!")
                    break  # Esce dal ciclo esterno

    if epochs_no_improve >= patience:
        break

# ==================== FINE TRAINING ==========================
print("\n✅ Training completato!")
print(f"📌 Miglior modello salvato in: {best_model_path}")

⬇️ Scarico il dataset da Roboflow...
loading Roboflow workspace...
loading Roboflow project...
✅ Dataset salvato in: /content/RoboAnimeProject-2
📊 Numero classi: 6
🖥️  Esecuzione su: cuda
📂 Carico immagini di training e validazione...
✅ Immagini train: 700 | valid: 200
🧠 Creo modello ResNet18 pre-addestrato...


The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.


🚀 Inizio training...


🔁 Epoch 1/20


📊 Train Loss: 1.5114 | Acc: 0.3771


📊 Val Loss: 1.4020 | Acc: 0.3750
💾 Model migliorato! Salvato.

🔁 Epoch 2/20


...

📊 Val Loss: 1.3558 | Acc: 0.5350
⏳ Nessun miglioramento (3/5)

🔁 Epoch 6/20


📊 Train Loss: 0.0666 | Acc: 0.9914


📊 Val Loss: 1.4010 | Acc: 0.5100
⏳ Nessun miglioramento (4/5)

🔁 Epoch 7/20


📊 Train Loss: 0.0536 | Acc: 0.9929


📊 Val Loss: 1.4807 | Acc: 0.4950
⏳ Nessun miglioramento (5/5)
🛑 Early stopping attivato!

✅ Training completato!
📌 Miglior modello salvato in: best_model.pt
